In [ ]:
import torch
import librosa
import os
import pandas as pd
import numpy as np

In [ ]:
!pip install muq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 126.7 MB/s eta 0:00:00


In [ ]:
from muq import MuQMuLan

# This will automatically fetch checkpoints from huggingface
device = 'cpu'
mulan = MuQMuLan.from_pretrained("OpenMuQ/MuQ-MuLan-large")
mulan = mulan.to(device).eval()

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

# Process audio

In [ ]:
import zipfile
import os

zip_files = ["Exp1.zip", "Exp2.zip", "Analysis.zip"]

extract_dir = "/content/" # You can change this if you want to extract elsewhere
os.makedirs(extract_dir, exist_ok=True)

for zip_file in zip_files:
    if os.path.exists(zip_file):
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted {zip_file} to {extract_dir}")
    else:
        print(f"Error: {zip_file} not found.")

Extracted Exp1.zip to /content/
Extracted Exp2.zip to /content/
Extracted Analysis.zip to /content/


In [ ]:
embeddings = []
stimuli_path = "/content/Exp1/Stimuli/"

for file in sorted(os.listdir(stimuli_path)):
    if file.endswith(".wav"):
        wav_path = os.path.join(stimuli_path, file)
        #Clap has already been trained on a sample rate of 48,000 so we should use what it knows already
        audio, sample_rate = librosa.load(wav_path, sr=24000)
        wav_tensor = torch.tensor(audio).unsqueeze(0).to(device)
        with torch.no_grad():
            embedding = mulan(wavs = wav_tensor)
        embeddings.append(embedding)

In [ ]:
audio_embeddings = torch.stack(embeddings)
audio_embeddings = audio_embeddings.squeeze(1)
audio_embeddings.shape

torch.Size([59, 512])

# Process text

In [ ]:
discrete_tags = ["happiness", "sadness", "anger", "tenderness", "fear"]
discrete_captions_perceived = ["I perceive this sound as " + tag for tag in discrete_tags]
print(discrete_captions_perceived)

discrete_captions_induced = ["This sound makes me feel " + tag for tag in discrete_tags]
print(discrete_captions_induced)

dimensional_tags = ["positive", "relaxed", "awake"]
dimensional_captions_perceived = ["I perceive this sound as " + tag for tag in dimensional_tags]
print(dimensional_captions_perceived)
dimensional_captions_induced = ["This sound makes me feel " + tag for tag in dimensional_tags]
print(dimensional_captions_induced)


['I perceive this sound as happiness', 'I perceive this sound as sadness', 'I perceive this sound as anger', 'I perceive this sound as tenderness', 'I perceive this sound as fear']
['This sound makes me feel happiness', 'This sound makes me feel sadness', 'This sound makes me feel anger', 'This sound makes me feel tenderness', 'This sound makes me feel fear']
['I perceive this sound as positive', 'I perceive this sound as relaxed', 'I perceive this sound as awake']
['This sound makes me feel positive', 'This sound makes me feel relaxed', 'This sound makes me feel awake']


In [ ]:
with torch.no_grad():
    tag_embeds = mulan(texts = dimensional_captions_induced)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
with torch.no_grad():
    tag_embeds_disc_i = mulan(texts = discrete_captions_induced)

In [ ]:
with torch.no_grad():
    tag_embeds_dim_p = mulan(texts = dimensional_captions_perceived)

In [ ]:
with torch.no_grad():
    tag_embeds_disc_p = mulan(texts = discrete_captions_perceived)

# Generate outputs

In [ ]:
sims = mulan.calc_similarity(audio_embeddings, tag_embeds)
print(sims.shape)

torch.Size([59, 3])


In [ ]:
sims_disc_i = mulan.calc_similarity(audio_embeddings, tag_embeds_disc_i)
print(sims_disc_i.shape)

torch.Size([59, 5])


In [ ]:
sims_dim_p = mulan.calc_similarity(audio_embeddings, tag_embeds_dim_p)
print(sims_dim_p.shape)

torch.Size([59, 3])


In [ ]:
sims_disc_p = mulan.calc_similarity(audio_embeddings, tag_embeds_disc_p)
print(sims_disc_p.shape)

torch.Size([59, 5])


## Load csv files and extract related columns

In [ ]:
IDim_path = '/content/Exp1/Data/IDim/'
#IDim: Indiced dimensional
IDim_responses = []

# All 65 responses concatenated to one data frame
all_dfs = []

for file in sorted(os.listdir(IDim_path)):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(IDim_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ['positive', 'relaxed', 'awake']
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

mean_positive_vector = []
mean_relaxed_vector = []
mean_awake_vector = []

num_rows = all_dfs[0].shape[0]

# 59 rows
for i in range(num_rows):
    current_row_positives = []
    current_row_relaxeds = []
    current_row_awakes = []

    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_positives.append(df.iloc[i]['positive'])
        current_row_relaxeds.append(df.iloc[i]['relaxed'])
        current_row_awakes.append(df.iloc[i]['awake'])

    # Calculate the mean for the current row across all files, for each column
    mean_positive_vector.append(np.mean(current_row_positives))
    mean_relaxed_vector.append(np.mean(current_row_relaxeds))
    mean_awake_vector.append(np.mean(current_row_awakes))

IDim_responses = {
    'positive_vector': mean_positive_vector,
    'relaxed_vector': mean_relaxed_vector,
    'awake_vector': mean_awake_vector
}

# One value per stimulus (mean from all participants rating)
print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of positive_vector: {len(IDim_responses['positive_vector'])}")
print(f"Length of relaxed_vector: {len(IDim_responses['relaxed_vector'])}")
print(f"Length of awake_vector: {len(IDim_responses['awake_vector'])}")


--- Mean Vectors (Mean across CSVs for each row position) ---
Length of positive_vector: 59
Length of relaxed_vector: 59
Length of awake_vector: 59


In [ ]:
PDim_path = '/content/Exp1/Data/PDim/'
#PDim: Percieved dimensional
PDim_responses = []

all_dfs = []

for file in sorted(os.listdir(PDim_path)):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(PDim_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ['positive', 'relaxed', 'awake']
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

mean_positive_vector = []
mean_relaxed_vector = []
mean_awake_vector = []

num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_positives = []
    current_row_relaxeds = []
    current_row_awakes = []

    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_positives.append(df.iloc[i]['positive'])
        current_row_relaxeds.append(df.iloc[i]['relaxed'])
        current_row_awakes.append(df.iloc[i]['awake'])

    # Calculate the mean for the current row across all files, for each column
    mean_positive_vector.append(np.mean(current_row_positives))
    mean_relaxed_vector.append(np.mean(current_row_relaxeds))
    mean_awake_vector.append(np.mean(current_row_awakes))

PDim_responses = {
    'positive_vector': mean_positive_vector,
    'relaxed_vector': mean_relaxed_vector,
    'awake_vector': mean_awake_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of positive_vector: {len(PDim_responses['positive_vector'])}")
print(f"Length of relaxed_vector: {len(PDim_responses['relaxed_vector'])}")
print(f"Length of awake_vector: {len(PDim_responses['awake_vector'])}")


--- Mean Vectors (Mean across CSVs for each row position) ---
Length of positive_vector: 59
Length of relaxed_vector: 59
Length of awake_vector: 59


In [ ]:
IDisc_path = '/content/Exp1/Data/IDisc/'
#IDisc: Induced discrete
IDisc_responses = []

all_dfs = []

for file in sorted(os.listdir(IDisc_path)):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(IDisc_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ["happiness", "sadness", "anger", "tenderness", "fear"]
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

mean_happiness_vector = []
mean_sadness_vector = []
mean_anger_vector = []
mean_tenderness_vector = []
mean_fear_vector = []



num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_happiness = []
    current_row_sadness = []
    current_row_anger = []
    current_row_tenderness = []
    current_row_fear = []


    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_happiness.append(df.iloc[i]['happiness'])
        current_row_sadness.append(df.iloc[i]['sadness'])
        current_row_anger.append(df.iloc[i]['anger'])
        current_row_tenderness.append(df.iloc[i]['tenderness'])
        current_row_fear.append(df.iloc[i]['fear'])


    # Calculate the mean for the current row across all files, for each column
    mean_happiness_vector.append(np.mean(current_row_happiness))
    mean_sadness_vector.append(np.mean(current_row_sadness))
    mean_anger_vector.append(np.mean(current_row_anger))
    mean_tenderness_vector.append(np.mean(current_row_tenderness))
    mean_fear_vector.append(np.mean(current_row_fear))

IDisc_responses = {
    'happiness_vector': mean_happiness_vector,
    'sadness_vector': mean_sadness_vector,
    'anger_vector': mean_anger_vector,
    'tenderness_vector': mean_tenderness_vector,
    'fear_vector': mean_fear_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of happiness_vector: {len(IDisc_responses['happiness_vector'])}")
print(f"Length of sadness_vector: {len(IDisc_responses['sadness_vector'])}")
print(f"Length of anger_vector: {len(IDisc_responses['anger_vector'])}")
print(f"Length of tenderness_vector: {len(IDisc_responses['tenderness_vector'])}")
print(f"Length of fear_vector: {len(IDisc_responses['fear_vector'])}")



--- Mean Vectors (Mean across CSVs for each row position) ---
Length of happiness_vector: 59
Length of sadness_vector: 59
Length of anger_vector: 59
Length of tenderness_vector: 59
Length of fear_vector: 59


In [ ]:
PDisc_path = '/content/Exp1/Data/PDisc/'
#PDisc: Percieved discrete
PDisc_responses = []

all_dfs = []

for file in sorted(os.listdir(PDisc_path)):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(PDisc_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ["happiness", "sadness", "anger", "tenderness", "fear"]
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

mean_happiness_vector = []
mean_sadness_vector = []
mean_anger_vector = []
mean_tenderness_vector = []
mean_fear_vector = []



num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_happiness = []
    current_row_sadness = []
    current_row_anger = []
    current_row_tenderness = []
    current_row_fear = []


    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_happiness.append(df.iloc[i]['happiness'])
        current_row_sadness.append(df.iloc[i]['sadness'])
        current_row_anger.append(df.iloc[i]['anger'])
        current_row_tenderness.append(df.iloc[i]['tenderness'])
        current_row_fear.append(df.iloc[i]['fear'])


    # Calculate the mean for the current row across all files, for each column
    mean_happiness_vector.append(np.mean(current_row_happiness))
    mean_sadness_vector.append(np.mean(current_row_sadness))
    mean_anger_vector.append(np.mean(current_row_anger))
    mean_tenderness_vector.append(np.mean(current_row_tenderness))
    mean_fear_vector.append(np.mean(current_row_fear))

PDisc_responses = {
    'happiness_vector': mean_happiness_vector,
    'sadness_vector': mean_sadness_vector,
    'anger_vector': mean_anger_vector,
    'tenderness_vector': mean_tenderness_vector,
    'fear_vector': mean_fear_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of happiness_vector: {len(PDisc_responses['happiness_vector'])}")
print(f"Length of sadness_vector: {len(PDisc_responses['sadness_vector'])}")
print(f"Length of anger_vector: {len(PDisc_responses['anger_vector'])}")
print(f"Length of tenderness_vector: {len(PDisc_responses['tenderness_vector'])}")
print(f"Length of fear_vector: {len(PDisc_responses['fear_vector'])}")



--- Mean Vectors (Mean across CSVs for each row position) ---
Length of happiness_vector: 59
Length of sadness_vector: 59
Length of anger_vector: 59
Length of tenderness_vector: 59
Length of fear_vector: 59


# Evaluate

In [ ]:
# Find the min and max values in the current sims tensor
old_min = sims.min()
old_max = sims.max()

# Define the new desired range
new_min = 1.0
new_max = 9.0

# Apply the min-max scaling formula
# Scaling CLAPs predictions to match the human ratings scale (1-9)
scaled_sims = ((sims - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims[:10]}\n")
print(f"Scaled sims min value: {scaled_sims.min():.4f}")
print(f"Scaled sims max value: {scaled_sims.max():.4f}\n")

human_ratings_tensor = torch.tensor([
    IDim_responses['positive_vector'],
    IDim_responses['relaxed_vector'],
    IDim_responses['awake_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 3)

print(f"Human ratings tensor shape: {human_ratings_tensor.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor[:10]}\n")

Scaled sims shape: torch.Size([59, 3])
Scaled sims (first 5 rows):
tensor([[3.3496, 5.6014, 7.0607],
        [3.7335, 5.5257, 7.5812],
        [3.9698, 4.6559, 6.5141],
        [3.2473, 4.4689, 6.6534],
        [4.9730, 5.2574, 7.9457],
        [5.3507, 6.5413, 9.0000],
        [3.7513, 4.7811, 7.3001],
        [3.9344, 6.4735, 7.7279],
        [3.8810, 6.9718, 7.9533],
        [2.9944, 5.9297, 6.7818]])

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 3])
Human ratings tensor (first 5 rows):
tensor([[4.4872, 4.8798, 4.5185],
        [4.7682, 5.3057, 4.7414],
        [5.3105, 5.7312, 5.0760],
        [5.2582, 5.2651, 5.4488],
        [3.8897, 3.9540, 5.3549],
        [4.3197, 3.8900, 5.5089],
        [4.9825, 5.1875, 5.5354],
        [5.0934, 4.8657, 5.8208],
        [4.7605, 4.8502, 5.4158],
        [5.2577, 4.8688, 5.7003]])



In [ ]:
# Find the min and max values in the current sims tensor
old_min_disc_i = sims_disc_i.min()
old_max_disc_i = sims_disc_i.max()

# Define the new desired range
new_min = 1.0
new_max = 9.0

# Apply the min-max scaling formula
scaled_sims_disc_i = ((sims_disc_i - old_min_disc_i) / (old_max_disc_i - old_min_disc_i)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims_disc_i.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims_disc_i[:10]}\n")
print(f"Scaled sims min value: {scaled_sims_disc_i.min():.4f}")
print(f"Scaled sims max value: {scaled_sims_disc_i.max():.4f}\n")

human_ratings_tensor_disc_i = torch.tensor([
    IDisc_responses['happiness_vector'],
    IDisc_responses['sadness_vector'],
    IDisc_responses['anger_vector'],
    IDisc_responses['tenderness_vector'],
    IDisc_responses['fear_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 5)

print(f"Human ratings tensor shape: {human_ratings_tensor_disc_i.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor_disc_i[:10]}\n")

Scaled sims shape: torch.Size([59, 5])
Scaled sims (first 5 rows):
tensor([[2.8518, 4.9617, 7.2381, 3.8343, 7.9868],
        [3.8356, 4.5759, 3.8879, 4.3856, 6.0775],
        [3.3891, 4.6150, 3.6356, 4.7423, 6.1308],
        [3.2091, 3.9148, 3.2761, 4.9812, 5.2618],
        [3.4923, 4.9421, 7.3166, 3.5787, 9.0000],
        [4.3154, 4.0183, 6.0950, 4.7943, 8.0286],
        [3.1380, 3.9895, 5.5160, 3.2913, 7.5740],
        [3.5243, 4.3860, 5.4207, 4.5220, 7.1585],
        [3.0705, 4.2693, 6.0801, 4.0590, 6.9545],
        [3.0055, 4.0194, 4.9943, 4.6266, 6.5338]])

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 5])
Human ratings tensor (first 5 rows):
tensor([[1.8202, 3.3418, 2.7948, 1.9789, 4.1430],
        [1.9091, 3.4459, 2.9059, 2.3785, 2.8558],
        [2.0644, 3.1847, 2.4182, 2.6371, 2.3356],
        [2.5321, 3.1900, 2.1509, 3.0583, 2.0938],
        [1.8391, 2.4733, 4.0144, 1.9902, 3.4942],
        [2.0029, 2.3036, 3.7795, 2.

In [ ]:
# Find the min and max values in the current sims tensor
old_min_disc_p = sims_disc_p.min()
old_max_disc_p = sims_disc_p.max()

# Define the new desired range
new_min = 1.0
new_max = 9.0

# Apply the min-max scaling formula
scaled_sims_disc_p = ((sims_disc_p - old_min_disc_p) / (old_max_disc_p - old_min_disc_p)) * (new_max - new_min) + new_min
print(scaled_sims_disc_p.shape)

print(f"Scaled sims shape: {scaled_sims_disc_p.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims_disc_p[:10]}\n")
print(f"Scaled sims min value: {scaled_sims_disc_p.min():.4f}")
print(f"Scaled sims max value: {scaled_sims_disc_p.max():.4f}\n")

human_ratings_tensor_disc_p = torch.tensor([
    PDisc_responses['happiness_vector'],
    PDisc_responses['sadness_vector'],
    PDisc_responses['anger_vector'],
    PDisc_responses['tenderness_vector'],
    PDisc_responses['fear_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 5)

print(f"Human ratings tensor shape: {human_ratings_tensor_disc_p.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor_disc_p[:10]}\n")

torch.Size([59, 5])
Scaled sims shape: torch.Size([59, 5])
Scaled sims (first 5 rows):
tensor([[3.9087, 4.6290, 9.0000, 4.0768, 8.0327],
        [5.7089, 4.7136, 4.2416, 4.7274, 5.6343],
        [4.8063, 4.9134, 4.6372, 4.3757, 6.1396],
        [5.3014, 3.7377, 3.8275, 4.3479, 4.7435],
        [4.1893, 4.7816, 8.3054, 3.4345, 8.1970],
        [5.5406, 3.5545, 6.0158, 4.2931, 7.0141],
        [3.4959, 3.5459, 6.5207, 2.9077, 7.0697],
        [4.5089, 3.9599, 6.4736, 4.2623, 6.8093],
        [3.7627, 3.1041, 5.4864, 3.6700, 5.5295],
        [3.9406, 3.3763, 5.6798, 3.9674, 6.1473]])

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 5])
Human ratings tensor (first 5 rows):
tensor([[1.5362, 4.4191, 5.1551, 2.0345, 5.6468],
        [1.7403, 4.8642, 4.2637, 2.5285, 3.9835],
        [2.0486, 5.5669, 3.0558, 2.9492, 2.8565],
        [2.4389, 4.8343, 2.1514, 3.5546, 3.0082],
        [1.5691, 3.6631, 5.0478, 1.9322, 4.7326],
        [2.3355

In [ ]:
# Find the min and max values in the current sims tensor
old_min_dim_p = sims_dim_p.min()
old_max_dim_p = sims_dim_p.max()

# Define the new desired range
new_min = 1.0
new_max = 9.0

# Apply the min-max scaling formula
scaled_sims_dim_p = ((sims_dim_p - old_min_dim_p) / (old_max_dim_p - old_min_dim_p)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims_dim_p.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims_dim_p[:10]}\n")
print(f"Scaled sims min value: {scaled_sims_dim_p.min():.4f}")
print(f"Scaled sims max value: {scaled_sims_dim_p.max():.4f}\n")

human_ratings_tensor_dim_p = torch.tensor([
    PDim_responses['positive_vector'],
    PDim_responses['relaxed_vector'],
    PDim_responses['awake_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 3)

print(f"Human ratings tensor shape: {human_ratings_tensor_dim_p.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor_dim_p[:10]}\n")

Scaled sims shape: torch.Size([59, 3])
Scaled sims (first 5 rows):
tensor([[3.3431, 5.3199, 8.2932],
        [3.7728, 6.3966, 8.5580],
        [4.2090, 6.3715, 8.2572],
        [3.6355, 6.1553, 7.6722],
        [4.6238, 5.3902, 8.0956],
        [4.8960, 5.7569, 8.2644],
        [3.0722, 4.7845, 7.8708],
        [3.5385, 5.9188, 8.0353],
        [2.4497, 4.7560, 6.5601],
        [2.7077, 5.1027, 7.2275]])

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 3])
Human ratings tensor (first 5 rows):
tensor([[3.5116, 4.4973, 3.4563],
        [4.1519, 4.9952, 3.2785],
        [4.6982, 5.4154, 4.2416],
        [5.2018, 5.2612, 4.9213],
        [3.1964, 3.4387, 3.9169],
        [3.9328, 3.8416, 4.5261],
        [4.5933, 4.8528, 4.9660],
        [4.5391, 4.6797, 5.4388],
        [4.2891, 4.7658, 4.2599],
        [4.6275, 4.4824, 5.1707]])



 # Comparison Metrics

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr # For Pearson correlation

# Comparison Method 1: Mean Absolute Error (MAE)
# MAE measures the average magnitude of the errors in a set of predictions, without considering their direction.
# Average of all absolute errors across all 59 stimuli and 3 dimensions.
# e.g An MAE of 1.5 means CLAP is typically ±1.5 points away from human ratings.
# Mean is calculated over the matrix so one value is outputted
mae = torch.mean(torch.abs(scaled_sims - human_ratings_tensor))
mae_disc_i = torch.mean(torch.abs(scaled_sims_disc_i - human_ratings_tensor_disc_i))
mae_disc_p = torch.mean(torch.abs(scaled_sims_disc_p - human_ratings_tensor_disc_p))
mae_dim_p = torch.mean(torch.abs(scaled_sims_dim_p - human_ratings_tensor_dim_p))

print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings (IDim) : {mae:.4f}")
print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings (IDisc) : {mae_disc_i:.4f}")
print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings (PDisc) : {mae_disc_p:.4f}")
print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings (PDim) : {mae_dim_p:.4f}")

# Comparison Method 2: Pearson Correlation Coefficient (per column)
# Pearson correlation measures the linear relationship between two sets of data.
# We'll calculate it for each of the three columns (dimensions).
# Which emotion dimensions does CLAP understand best?

#IDim
correlation_positive, _ = pearsonr(scaled_sims[:, 0].detach().numpy(), human_ratings_tensor[:, 0].numpy())
correlation_relaxed, _ = pearsonr(scaled_sims[:, 1].detach().numpy(), human_ratings_tensor[:, 1].numpy())
correlation_awake, _ = pearsonr(scaled_sims[:, 2].detach().numpy(), human_ratings_tensor[:, 2].numpy())

#PDim
correlation_positive_dim_p, _ = pearsonr(scaled_sims_dim_p[:, 0].detach().numpy(), human_ratings_tensor_dim_p[:, 0].numpy())
correlation_relaxed_dim_p, _ = pearsonr(scaled_sims_dim_p[:, 1].detach().numpy(), human_ratings_tensor_dim_p[:, 1].numpy())
correlation_awake_dim_p, _ = pearsonr(scaled_sims_dim_p[:, 2].detach().numpy(), human_ratings_tensor_dim_p[:, 2].numpy())

# IDisc
correlation_happiness, _ = pearsonr(scaled_sims_disc_i[:, 0].detach().numpy(), human_ratings_tensor_disc_i[:, 0].numpy())
correlation_sadness, _ = pearsonr(scaled_sims_disc_i[:, 1].detach().numpy(), human_ratings_tensor_disc_i[:, 1].numpy())
correlation_anger, _ = pearsonr(scaled_sims_disc_i[:, 2].detach().numpy(), human_ratings_tensor_disc_i[:, 2].numpy())
correlation_tenderness, _ = pearsonr(scaled_sims_disc_i[:, 3].detach().numpy(), human_ratings_tensor_disc_i[:, 3].numpy())
correlation_fear, _ = pearsonr(scaled_sims_disc_i[:, 4].detach().numpy(), human_ratings_tensor_disc_i[:, 4].numpy())

#PDisc
correlation_happiness_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 0].detach().numpy(), human_ratings_tensor_disc_p[:, 0].numpy())
correlation_sadness_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 1].detach().numpy(), human_ratings_tensor_disc_p[:, 1].numpy())
correlation_anger_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 2].detach().numpy(), human_ratings_tensor_disc_p[:, 2].numpy())
correlation_tenderness_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 3].detach().numpy(), human_ratings_tensor_disc_p[:, 3].numpy())
correlation_fear_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 4].detach().numpy(), human_ratings_tensor_disc_p[:, 4].numpy())

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings) IDim:")
print(f"  Positive Dimension: {correlation_positive:.4f}")
print(f"  Relaxed Dimension:  {correlation_relaxed:.4f}")
print(f"  Awake Dimension:    {correlation_awake:.4f}\n")

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings) PDim:")
print(f"  Positive Dimension: {correlation_positive_dim_p:.4f}")
print(f"  Relaxed Dimension:  {correlation_relaxed_dim_p:.4f}")
print(f"  Awake Dimension:    {correlation_awake_dim_p:.4f}\n")

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings) IDisc:")
print(f"  Happiness Dimension: {correlation_happiness:.4f}")
print(f"  Sadness Dimension:  {correlation_sadness:.4f}")
print(f"  Anger Dimension:    {correlation_anger:.4f}")
print(f"  Tenderness Dimension:    {correlation_tenderness:.4f}")
print(f"  Fear Dimension:    {correlation_fear:.4f}\n")

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings) PDisc:")
print(f"  Happiness Dimension: {correlation_happiness_disc_p:.4f}")
print(f"  Sadness Dimension:  {correlation_sadness_disc_p:.4f}")
print(f"  Anger Dimension:    {correlation_anger_disc_p:.4f}")
print(f"  Tenderness Dimension:    {correlation_tenderness_disc_p:.4f}")
print(f"  Fear Dimension:    {correlation_fear_disc_p:.4f}\n")



average_correlation = (correlation_positive + correlation_relaxed + correlation_awake) / 3
average_correlation_disc_i = (correlation_happiness + correlation_sadness + correlation_anger + correlation_tenderness + correlation_fear) / 5
average_correlation_disc_p = (correlation_happiness_disc_p + correlation_sadness_disc_p + correlation_anger_disc_p + correlation_tenderness_disc_p + correlation_fear_disc_p) / 5
average_correlation_dim_p = (correlation_positive_dim_p + correlation_relaxed_dim_p + correlation_awake_dim_p) / 3

print(f"  Average Correlation (IDim): {average_correlation:.4f}")
print(f"  Average Correlation (IDisc): {average_correlation_disc_i:.4f}")
print(f"  Average Correlation (PDisc): {average_correlation_disc_p:.4f}")
print(f"  Average Correlation (PDim): {average_correlation_dim_p:.4f}")

Mean Absolute Error (MAE) between scaled_sims and human_ratings (IDim) : 1.5562
Mean Absolute Error (MAE) between scaled_sims and human_ratings (IDisc) : 2.0035
Mean Absolute Error (MAE) between scaled_sims and human_ratings (PDisc) : 1.6638
Mean Absolute Error (MAE) between scaled_sims and human_ratings (PDim) : 1.7383
Pearson Correlation Coefficients (between scaled_sims and human_ratings) IDim:
  Positive Dimension: 0.2551
  Relaxed Dimension:  -0.3117
  Awake Dimension:    0.2681

Pearson Correlation Coefficients (between scaled_sims and human_ratings) PDim:
  Positive Dimension: 0.2952
  Relaxed Dimension:  0.0191
  Awake Dimension:    -0.4583

Pearson Correlation Coefficients (between scaled_sims and human_ratings) IDisc:
  Happiness Dimension: 0.1790
  Sadness Dimension:  0.2930
  Anger Dimension:    0.0406
  Tenderness Dimension:    0.4086
  Fear Dimension:    0.2479

Pearson Correlation Coefficients (between scaled_sims and human_ratings) PDisc:
  Happiness Dimension: -0.0021


In [ ]:
# Comparison Method 2: Mean Absolute Percentage Error (MAPE)
# MAPE measures the accuracy of a forecasting method in terms of percentage.
# Formula: MAPE = (1/n) * sum(|(Actual - Forecast) / Actual|) * 100%
# On average, what percentage do CLAP's predictions deviate from human ratings?

# Calculate the absolute percentage error for each element
# Since human responses are between 1 and 9, division by zero is not a concern.
absolute_percentage_error = torch.abs((human_ratings_tensor - scaled_sims) / human_ratings_tensor) * 100
absolute_percentage_error_disc_i = torch.abs((human_ratings_tensor_disc_i - scaled_sims_disc_i) / human_ratings_tensor_disc_i) * 100
absolute_percentage_error_disc_p = torch.abs((human_ratings_tensor_disc_p - scaled_sims_disc_p) / human_ratings_tensor_disc_p) * 100
absolute_percentage_error_dim_p = torch.abs((human_ratings_tensor_dim_p - scaled_sims_dim_p) / human_ratings_tensor_dim_p) * 100

# Calculate the mean of these percentage errors
mape = torch.mean(absolute_percentage_error)
mape_disc_i = torch.mean(absolute_percentage_error_disc_i)
mape_disc_p = torch.mean(absolute_percentage_error_disc_p)
mape_dim_p = torch.mean(absolute_percentage_error_dim_p)

print(f"Mean Absolute Percentage Error (MAPE) IDim: {mape:.2f}%")
print(f"Mean Absolute Percentage Error (MAPE) IDisc: {mape_disc_i:.2f}%")
print(f"Mean Absolute Percentage Error (MAPE) PDisc: {mape_disc_p:.2f}%")
print(f"Mean Absolute Percentage Error (MAPE) PDim: {mape_dim_p:.2f}%\n")

# Comparison Method 3: Root Mean Squared Error (RMSE)
# RMSE measures the square root of the average of the squared differences between predicted and actual values.
# It gives a relatively high weight to large errors.
# Formula: RMSE = sqrt(mean((Actual - Forecast)^2))
# How far, on average, are CLAP’s predictions from human ratings—while heavily punishing big errors?

# Calculate the squared differences
squared_differences = (human_ratings_tensor - scaled_sims)**2
squared_differences_disc_i = (human_ratings_tensor_disc_i - scaled_sims_disc_i)**2
squared_differences_disc_p = (human_ratings_tensor_disc_p - scaled_sims_disc_p)**2
squared_differences_dim_p = (human_ratings_tensor_dim_p - scaled_sims_dim_p)**2


# Calculate the mean of the squared differences (Mean Squared Error - MSE)
mse = torch.mean(squared_differences)
mse_disc_i = torch.mean(squared_differences_disc_i)
mse_disc_p = torch.mean(squared_differences_disc_p)
mse_dim_p = torch.mean(squared_differences_dim_p)

# Calculate the square root to get RMSE
rmse = torch.sqrt(mse)
rmse_disc_i = torch.sqrt(mse_disc_i)
rmse_disc_p = torch.sqrt(mse_disc_p)
rmse_dim_p = torch.sqrt(mse_dim_p)

print(f"Root Mean Squared Error (RMSE) IDim: {rmse:.4f}\n")
print(f"Root Mean Squared Error (RMSE) IDisc: {rmse_disc_i:.4f}\n")
print(f"Root Mean Squared Error (RMSE) PDisc: {rmse_disc_p:.4f}\n")
print(f"Root Mean Squared Error (RMSE) PDim: {rmse_dim_p:.4f}\n")

from sklearn.metrics import r2_score

# Comparison Method 4: R-Squared
# Measures how well CLAP's predictions explain the variance in human ratings.
# What percentage of the changes in human ratings can be predicted by CLAP's model? (1=perfect prediction)

# R-squared IDim
print("\nR-squared scores (IDim):")
r2_valence = r2_score(human_ratings_tensor[:, 0].numpy(), scaled_sims[:, 0].detach().numpy())
print("  valence =", r2_valence)
r2_tension = r2_score(human_ratings_tensor[:, 1].numpy(), scaled_sims[:, 1].detach().numpy())
print("  tension =", r2_tension)
r2_energy = r2_score(human_ratings_tensor[:, 2].numpy(), scaled_sims[:, 2].detach().numpy())
print("  energy =", r2_energy)

# R-squared IDisc
print("\nR-squared scores (IDisc):")
r2_happiness_disc_i = r2_score(human_ratings_tensor_disc_i[:, 0].numpy(), scaled_sims_disc_i[:, 0].detach().numpy())
print("  happiness =", r2_happiness_disc_i)
r2_sadness_disc_i = r2_score(human_ratings_tensor_disc_i[:, 1].numpy(), scaled_sims_disc_i[:, 1].detach().numpy())
print("  sadness =", r2_sadness_disc_i)
r2_anger_disc_i = r2_score(human_ratings_tensor_disc_i[:, 2].numpy(), scaled_sims_disc_i[:, 2].detach().numpy())
print("  anger =", r2_anger_disc_i)
r2_tenderness_disc_i = r2_score(human_ratings_tensor_disc_i[:, 3].numpy(), scaled_sims_disc_i[:, 3].detach().numpy())
print("  tenderness =", r2_tenderness_disc_i)
r2_fear_disc_i = r2_score(human_ratings_tensor_disc_i[:, 4].numpy(), scaled_sims_disc_i[:, 4].detach().numpy())
print("  fear =", r2_fear_disc_i)

# R-squared PDim
print("\nR-squared scores (PDim):")
r2_valence_dim_p = r2_score(human_ratings_tensor_dim_p[:, 0].numpy(), scaled_sims_dim_p[:, 0].detach().numpy())
print("  valence =", r2_valence_dim_p)
r2_tension_dim_p = r2_score(human_ratings_tensor_dim_p[:, 1].numpy(), scaled_sims_dim_p[:, 1].detach().numpy())
print("  tension =", r2_tension_dim_p)
r2_energy_dim_p = r2_score(human_ratings_tensor_dim_p[:, 2].numpy(), scaled_sims_dim_p[:, 2].detach().numpy())
print("  energy =", r2_energy_dim_p)

# R-squared PDisc
print("\nR-squared scores (PDisc):")
r2_happiness_disc_p = r2_score(human_ratings_tensor_disc_p[:, 0].numpy(), scaled_sims_disc_p[:, 0].detach().numpy())
print("  happiness =", r2_happiness_disc_p)
r2_sadness_disc_p = r2_score(human_ratings_tensor_disc_p[:, 1].numpy(), scaled_sims_disc_p[:, 1].detach().numpy())
print("  sadness =", r2_sadness_disc_p)
r2_anger_disc_p = r2_score(human_ratings_tensor_disc_p[:, 2].numpy(), scaled_sims_disc_p[:, 2].detach().numpy())
print("  anger =", r2_anger_disc_p)
r2_tenderness_disc_p = r2_score(human_ratings_tensor_disc_p[:, 3].numpy(), scaled_sims_disc_p[:, 3].detach().numpy())
print("  tenderness =", r2_tenderness_disc_p)
r2_fear_disc_p = r2_score(human_ratings_tensor_disc_p[:, 4].numpy(), scaled_sims_disc_p[:, 4].detach().numpy())
print("  fear =", r2_fear_disc_p)

average_r2 = (r2_valence + r2_energy + r2_tension) /3
average_r2_dim_p = (r2_valence_dim_p + r2_energy_dim_p + r2_tension_dim_p) /3
average_r2_disc_i = (r2_happiness_disc_i + r2_sadness_disc_i + r2_anger_disc_i + r2_tenderness_disc_i + r2_fear_disc_i) / 5
average_r2_disc_p = (r2_happiness_disc_p + r2_sadness_disc_p + r2_anger_disc_p + r2_tenderness_disc_p + r2_fear_disc_p) / 5

print("\nAverage R-squared scores:")
print("IDim: ", average_r2)
print("PDim: ", average_r2_dim_p)
print("IDisc: ", average_r2_disc_i)
print("PDisc: ", average_r2_disc_p)


Mean Absolute Percentage Error (MAPE) IDim: 34.74%
Mean Absolute Percentage Error (MAPE) IDisc: 90.46%
Mean Absolute Percentage Error (MAPE) PDisc: 65.05%
Mean Absolute Percentage Error (MAPE) PDim: 36.46%

Root Mean Squared Error (RMSE) IDim: 1.9601

Root Mean Squared Error (RMSE) IDisc: 2.4271

Root Mean Squared Error (RMSE) PDisc: 1.9822

Root Mean Squared Error (RMSE) PDim: 2.1317


R-squared scores (IDim):
  valence = -2.2317891120910645
  tension = -2.3019065856933594
  energy = -5.570822238922119

R-squared scores (IDisc):
  happiness = -0.47385942935943604
  sadness = -6.284795761108398
  anger = -4.6211676597595215
  tenderness = -6.076670169830322
  fear = -22.617210388183594

R-squared scores (PDim):
  valence = -2.428204298019409
  tension = -0.6092023849487305
  energy = -5.028651714324951

R-squared scores (PDisc):
  happiness = -2.209482431411743
  sadness = 0.06781452894210815
  anger = -3.065157413482666
  tenderness = -1.7948229312896729
  fear = -3.6572275161743164

